Fully Homomorphic Encryption
==========================

This is a Fully Homomorphic Encryption system whose security is based on Ring-LWE.
This system is an implementation of the Fan-Vercauteren FHE mechanism using Gentry's bootstrapping

&nbsp;
&nbsp;
&nbsp;

Imports
-----------

In [3]:
import unittest
import random
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler

In [4]:
def param_gen(sec, n):
    '''
    Generates appropriate parameters according to FHE standards
    '''
    table = [[29, 21, 16], [56, 39, 31], [111, 77, 60], [220, 154, 120], [440, 307, 239], [880, 612, 478]]
    k = 0 if sec==128 else 1 if sec==192 else 2
    m = log(n,2)-10
    return table[m][k]

FHE Class
---------

In [9]:
class SHE():
    """
    Constructs a (Leveled) Fully Homomorphic Encryption Enviroment
    
    Constructs a new encryption environment who can be used as
    an LWE_PKE object but also provides addition and multiplication
    on encrypted ciphertexts and has security based on R-LWE
    
    Parameters
    ----------
    sec_lambda : int
        security parameter which defines dimension, defaults to 512
    """
    def __init__(self, sec_lambda=128, n=1024, error_dist=None):
        k = param_gen(sec_lambda, n)
        self.q = randint(2^k, 2^k+1)
        
        #TODO introduce balance function for ring elements
        R.<x> = PolynomialRing(Zmod(self.q))
        self.R = QuotientRing(R, R.ideal(x^n + 1)) #univariate polynomial ring with f(x)=x^n+1 
        
        self.chi = error_dist
        if not error_dist:
            sigma=8/sqrt(2*pi)
            self.chi = DiscreteGaussianDistributionPolynomialSampler(self.R, n, sigma)
        
        self._sk = self.SecretKeyGen()
        self._pk = self.PublicKeyGen(self._sk)
        
    def SecretKeyGen(self):
        """
        Generates a (monic polynomial) secret key
        
        Parameters
        ----------
        R : Ring to generate monic key from
        
        Returns
        -------
        secret_key
            a monic polynomial in R
        """
        ZZx = PolynomialRing(ZZ, 'x')
        r_sam = self.R.random_element().lift()
        monic = ZZx(r_sam)%2
        return self.R(monic)
    
    def PublicKeyGen(self, sk):
        """
        Generates a public key pair of polynomials
        
        Parameters
        ----------
        sk : secret key to generate public key from
        
        Returns
        -------
        public_key
            a pair of two polynomials in R (a, b) where b is some random polynomial
            and a is: b modified by the secret key and adjusted by some error
        """
        a = self.R.random_element()
        e = self.chi()
        return (self.R(-a*sk+e), a)
    
    
class FHE(SHE):
    '''
    Implements a true fully homomorphic encryption enviroment using bootstrapping
    This class only implements parameter selection and bootstrapping should be done externally.
    '''
    def __init(self):
        log_delta = 1.8/(sec_lambda+110)
        Hf = 1 #for simplicity assume parameterized family x^n+1
        t = 2 #plaintext space
        h = 63 #hamming weight
        alpha, beta = 3.8, 9.2 #with e=2^-64
        d=2^10 #set d=2^k (and q=2^n)
        L_min = ceil(log(t * 2 * (Hf*h + 1) + 0.5, 2))
        

        top = log(4*alpha*beta*t^(L_min-1),2) + (2*L_min+1)*log(d,2)
        bot = 2*sqrt(d*log_delta)
        n=ceil((top/bot)^2)
        self.q = 2^n #coefficient modulus
        
        R.<x> = PolynomialRing(Zmod(self.q))
        self.R = QuotientRing(R, R.ideal(x^d + 1)) #univariate polynomial ring with f(x)=x^n+1 
        
        sigma=ceil((alpha*self.q)/2^(2*sqrt(d*log_delta*n)))      
        self.chi = DiscreteGaussianDistributionPolynomialSampler(R, d, sigma)
        
        self._sk = self.SecretKeyGen()
        self._pk = self.PublicKeyGen(self._sk)
        

In [10]:
fhe = FHE(128)

2*xbar^1023 + 5*xbar^1022 + xbar^1020 + 2*xbar^1019 + xbar^1018 + 536870910*xbar^1017 + 3*xbar^1016 + 5*xbar^1015 + 2*xbar^1014 + 536870912*xbar^1013 + 536870908*xbar^1011 + 4*xbar^1010 + 536870912*xbar^1009 + 536870910*xbar^1008 + 3*xbar^1007 + 5*xbar^1006 + 3*xbar^1005 + 536870912*xbar^1004 + xbar^1003 + 7*xbar^1002 + 536870909*xbar^1001 + xbar^1000 + 536870911*xbar^999 + 536870910*xbar^998 + xbar^997 + xbar^996 + 2*xbar^995 + 3*xbar^994 + 2*xbar^993 + 536870910*xbar^991 + 3*xbar^990 + 536870911*xbar^989 + 536870901*xbar^988 + 536870912*xbar^987 + 536870910*xbar^986 + 536870912*xbar^985 + 4*xbar^984 + 4*xbar^983 + 536870910*xbar^982 + 5*xbar^981 + 536870912*xbar^980 + 536870912*xbar^979 + 536870912*xbar^978 + 536870911*xbar^977 + 536870912*xbar^976 + 536870911*xbar^975 + 536870910*xbar^974 + 536870911*xbar^973 + xbar^970 + 3*xbar^969 + xbar^968 + 536870909*xbar^967 + 536870910*xbar^966 + 536870908*xbar^965 + 6*xbar^964 + 2*xbar^963 + 536870910*xbar^962 + 536870909*xbar^961 + 53687091

Unit Tests
--------------

In [3]:
class testFHE(unittest.TestCase):
    def test_secretKey(self):
        n = randint(1, 512)
        fhe = FHE(n)
        key = fhe._sk.lift()
        coef = key.coefficients(sparse=False)
        for i in coef:
            if abs(i) not in [1, -1, 0]:
                self.fail("non-monic secret key")
        self.assertLessEqual(key.degree(), n)
        
        

if __name__ == '__main__':
    unittest.main(argv=['-v'], verbosity=2, exit=False)

test_secretKey (__main__.testFHE) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.034s

OK
